In [3]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import time
import shutil
import random
import csv
import os,sys
import ipywidgets as widgets
from ipywidgets import Layout, interact, interact_manual, fixed
import IPython.display as display
from IPython.display import clear_output
import pandas as pd
from tabulate import tabulate
%load_ext autoreload
%autoreload 2

#main_dir = os.path.normpath(os.getcwd() + os.sep + os.pardir + os.sep + os.pardir)
main_dir = os.getcwd()
print(main_dir)

D:\3104Project\ict3104-team11-2022


In [4]:
def on_button_clicked(b):
    batch_size_value = int(batch_size.value)
    epoch_value = epoch.value
    argv_dict["batch_size"] = batch_size_value
    argv_dict["epoch"] = epoch_value
    print("Values Set: ")
    print("Batch Size: ", batch_size_value)
    print("Epoch: ", epoch_value)

def on_clear_clicked(b):
    clear_output(wait=False)

def sidebyside(list1):
  side2side = widgets.HBox(list1)
  display.display(side2side)
  return list1

def batchButtonClick(side2side):
  button.on_click(on_button_clicked)
  clear.on_click(on_clear_clicked)

In [9]:
argv_dict = {}
flag = False

epoch = widgets.IntSlider(
    value=20,
    min=5,
    max=100,
    step=5,
    description='Epoch:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

batch_size = widgets.Dropdown(
    value = '2',
    placeholder='Choose Batch Size',
    options=['2', '4', '8', '16', '32', '64'],
    description='batch_size:',
    ensure_option=True,
    disabled=False
)


button = widgets.Button(description="Add",icon='check', command=on_button_clicked)
clear = widgets.Button(description="Clear",icon='clear', command=on_clear_clicked)

In [10]:
batchButtonClick(sidebyside([batch_size, epoch, button, clear]))

Values Set: 
Batch Size:  8
Epoch:  20


In [11]:
print(argv_dict)
print(argv_dict["batch_size"])

{'batch_size': 8, 'epoch': 20}
8


In [ ]:
#To get the current directory and set the path of the folder to the data folder
#To get the current directory and set the path of the folder to the data folder
def getDataFolderVideoContent():
    current_dir = str(os.getcwd())
    data_folder = current_dir + "\\Datasets\\Video"
    data_folder_content = []

    #Loop through to get all the contents inside the data folder
    with os.scandir(data_folder) as entries:
        for entry in entries:
            data_folder_content.append(entry.name)
    
    return data_folder_content

#Dropdown to display all videos in data folder
video_dropdown = widgets.Dropdown(
    options = getDataFolderVideoContent(),
    description = 'Videos:',
    disabled = False,
)

display(video_dropdown)

In [ ]:
#Set the selected video to the iPyWidget video function and display the video playback
selected_video = Video.from_file('Datasets/Video/' + video_dropdown.value)
selected_video